# Reed-Solomon codebook generating Notebook - Colab version

This notebook runs on [Google Colab](https://colab.research.google.com/github/CaiGroup/UntanglingBarcodes/blob/master/codebook_generation/get_RS_codebooks/colab/gen_RS_q11k7_code.jl.ipynb) to generate Reed-Solomon codebooks for seqFISH experiments

To run this notebook, you will need to change to the Julia runtime environment. To do that select from the drop down menu:

Runtime > Change runtime type

In the "Change runtime type"  prompt window, select "Julia" (not julia x.xx.xx) from the Runtime type drop-down menu. Click Save.

In [ ]:
using Pkg
# This notebook does not work with new versions of Nemo
Pkg.add(name="Nemo", version="0.36.2");

In [ ]:
using Nemo

# Introduction

This notebook shows how to generate codebooks for seqFISH experiments using [Reed-Solomon Codes](https://en.wikipedia.org/wiki/Reed%E2%80%93Solomon_error_correction). Reed-Solomon are part of a special class of error-correcting codes called [Maximum Distance Separable codes](https://en.wikipedia.org/wiki/Singleton_bound#MDS_codes) (MDS code) which achieve equality in the [Singleton bound](https://en.wikipedia.org/wiki/Singleton_bound). This means that MDS codes acheive the maximum possible extra difference between their codewords from every redundant parity check symbol, and gain the the most possible robustless to error for the increased cost of encoding information with more symbols.

The number of codewords of a given weight in an MDS code weight is given by the the expression 

$(q-1)\binom{w}{n}\sum_{i=0}^{w-d}(-1)^i \binom{i}{w-1}q^{w-d-i}$

(Macwilliams and Sloan)


## Set Reed-Solomon code parameters

In [ ]:
q = 11
k = 7

In [ ]:
function def_RS_code(_q :: Int64, deg :: Int64, nmk :: Int64)
    global q = _q
    global k = (q-1) - nmk
    F, α = FiniteField(q, deg, "α")
    R, x = PolynomialRing(F, "x")
    RR =  ResidueRing(R, x^(q^deg-1)-1)
    gp = 1
    for i = 1:nmk
        gp = gp*RR(x - α^i)
    end
    return RR, R, gp, x, α
end


function cvt_fq_nmod_2_int(x::fq_nmod)
    if iszero(x)
        return 0
    end
    for i = 1:(q-1)
        if iszero(i+x)
            return q-i
        end
    end 
end

function get_cw_array(cw)
    coeffs = Array{Union{Int8, Nothing}}(nothing, q-1)
    for i = 0:(q-2) #9
        coeffs[i+1] = cvt_fq_nmod_2_int(coeff(cw.data,i))
    end
    
    return coeffs
end


RR, R, gp, x, α = def_RS_code(q,1,q-1-k)

We can pring the generating polynomial

In [ ]:
gp

In [ ]:
# function convert codeword from specialized math objects to an Int Array
get_cw(i) = get_cw_array(gp*R(collect(Tuple(i)))) 

### Get the codewords

In [ ]:
@time q11_k7 = map(get_cw, CartesianIndices(Tuple(fill(q,k))));

In [ ]:
length(q11_k7)

In [ ]:
q11_k7 = reshape(q11_k7, length(q11_k7))

Let's count how many codewords there are of each weight. Compare to the output of Make_RS_Code_Table_expanded.jl

In [ ]:
weights = [sum(map(cw -> sum(.~iszero.(cw)), q11_k7) .== w) for w in 0:10]
weights

In [ ]:
cb = []
weights_en = [1,0,0,0,0,0,0,0,0,0,0]

w4_cws = []
w5_cws = []
w6_cws = []
niters = 0
li1 = -1
for cw in q11_k7 #CartesianIndices(Tuple(fill(q,k)))
    weight = sum(cw .!= 0)
    if weight == 4
        push!(w4_cws, cw)
    end
    if weight == 5
        push!(w5_cws, cw)
    end
    if weight == 6
        w6_cws = []
    end
    for w = 3:10
        if weight == w
            weights_en[w+1] += 1
        end
    end
end

### Save the codebooks containing codewords of each weight

In [ ]:
using DelimitedFiles
open("RS_q"*string(q)*"_k"*string(k)*"_w4cb.csv", "w") do io
    writedlm(io, hcat(w4_cws...)',",")
end

In [ ]:
using DelimitedFiles
open("RS_q"*string(q)*"_k"*string(k)*"_w5cb.csv", "w") do io
    writedlm(io, hcat(w5_cws...)',",")
end

In [ ]:
using DelimitedFiles
open("RS_q"*string(q)*"_k"*string(k)*"_w6cb.csv", "w") do io
    writedlm(io, hcat(w6_cws...)',",")
end

In [ ]:
w4_cws

### Get the parity check matrix

In [ ]:
H = [cvt_fq_nmod_2_int((α^(i[1]))^i[2]) for i in CartesianIndices((q-k-1, q-1))]
H

### Check that the matrix product of the parity check matrix and all codewords is 0

In [ ]:
all(map(cw -> all(iszero.(H*cw .% q)), w4_cws))

In [ ]:
all(map(cw -> all(iszero.(H*cw .% q)), w5_cws))

In [ ]:
all(map(cw -> all(iszero.(H*cw .% q)), w6_cws))

### Save the parity check matrix

In [ ]:
open("RS_q"*string(q)*"_k"*string(k)*"_H.csv", "w") do io
    writedlm(io, H,",")
end